# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [7]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [8]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [9]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [10]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [11]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [12]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [13]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [14]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [15]:
from typing import List

def multi_user_completion(
    system_prompt: str,
    user_prompts: List[str],
    model: str = MODEL,
    **kwargs
) -> str:
    messages = [{"role": "system", "content": system_prompt}]
    for p in user_prompts:
        messages.append({"role": "user", "content": p})
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        **kwargs
    )
    return response.choices[0].message.content


In [16]:

def get_links(url, extra_prompts: List[str] = None):
    website = Website(url)
    # İlk prompt her zaman link seçim sistemi
    prompts = [get_links_user_prompt(website)]
    # İsteğe bağlı ek promptlar
    if extra_prompts:
        prompts.extend(extra_prompts)

    content = multi_user_completion(
        system_prompt=link_system_prompt,
        user_prompts=prompts,
        model=MODEL,
        response_format={"type": "json_object"}
    )
    return json.loads(content)

In [17]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/moonshotai/Kimi-K2-Instruct',
 '/HuggingFaceTB/SmolLM3-3B',
 '/THUDM/GLM-4.1V-9B-Thinking',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/mistralai/Devstral-Small-2507',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/kontext-community/kontext-relight',
 '/spaces/FunAudioLLM/ThinkSound',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/hackaprompt/Pliny_HackAPrompt_Dataset',
 '/datasets/HuggingFaceTB/smoltalk2',
 '/datasets/cais/hle',
 '/datasets/GPUMODE/kernelbot-data',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '

In [18]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [19]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [20]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M

In [22]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information. Ayrıca bu yazılar türkce olsun"


In [36]:
# def get_brochure_user_prompt(company_name, url):
#     user_prompt = f"You are looking at a company called: {company_name}\n"
#     user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
#     user_prompt += get_all_details(url)
#     user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
#     return user_prompt

def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += "⚠️ Please write the brochure in English, regardless of the website’s original language.\n\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000]
    return user_prompt


In [34]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmoonshotai/Kimi-K2-Instruct\nUpdated\n20 minutes ago\n•\n19.3k\n•\n1.04k\nHuggingFaceTB/SmolLM3-3B\nUpdated\n5 days ago\n•\n27.1k\n•\n452\nTHUDM/GLM-4.1V-9B-Thinking\nUpdated\n7 days ago\n•\n35.7k\n•\n597\nblack-forest-labs/FLUX.1-Kontext-dev\nUpdated\n17 days ago\n•\n254k\n•\n1.65k\nmistralai/Devstral-Small-2507\nUpdated\n5 days ago\n•\n7.33k\n•\n205\nBrowse 1M+ models\nSpaces\nRunning\n1

In [37]:
def create_brochure(company_name: str, url: str):
    # 1. Brochure içeriği için öncelikle sayfa detaylarını al
    details = get_all_details(url)
    
    # 2. İki aşamalı user prompt listesi hazırla
    brochure_prompts = [
        details,
        "Bu bilgileri kullanarak kısa, esprili bir broşür oluştur ve Markdown formatında döndür. Ayrıca metni ingilizce yaz yaz."
    ]
    
    # 3. multi_user_completion ile çağır
    brochure_md = multi_user_completion(
        system_prompt=system_prompt,
        user_prompts=brochure_prompts,
        model=MODEL
    )
    
    # 4. Çıktıyı göster
    display(Markdown(brochure_md))


In [38]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'company social media', 'url': 'https://twitter.com/huggingface'}, {'type': 'company social media', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


```markdown
# Welcome to Hugging Face! 🤗

### The AI Community Building the Future

At Hugging Face, we’re not just hugging data; we’re embracing the future of AI! Here’s a sneak peek into our world, where models are plentiful, datasets abound, and the coffee is always fresh!

---

## What Do We Do? 🤔

- **Models Galore**: We’ve got over 1 million models. No, that’s not a typo! Whether you need text, images, or even sound, we’ve got models that can do it all (well, almost!).

- **Datasets, Datasets, Datasets**: Need a dataset? We’ve got 250,000+! That’s right; you can train your AI on unicorns, aliens, or whatever suits your fancy. 

- **Spaces for All**: Dive into our Spaces! Run applications that generate videos, relay images, and give life to your wildest AI dreams. 

---

## Our Culture 🌍

Here at Hugging Face, it’s all about collaboration, innovation, and, of course, a healthy dose of fun. We believe:
- **You can teach an old model new tricks!** 
- **Coffee is our secret sauce** (shh, don’t tell the engineers).
- **Work hard, play harder**—because who doesn’t want to brainstorm while playing chess with an AI?

---

## Join Us! 🚀

Feeling inspired? We’re always on the lookout for bright minds to join our team! With positions ranging from Software Engineers to AI Researchers, we promise you won’t be bored (unless you count model training as boring)!

### Current Openings:
- **Machine Learning Engineers**
- **Data Scientists**
- **AI Enthusiasts (must love coffee)**

---

## Our Customers 🌟

Join the ranks of over 50,000 organizations, including giants like Google, Microsoft, and Amazon! With Hugging Face, you’ll be in great company—literally!

---

**Ready to get your AI groove on?** 

Explore, collaborate, and innovate with Hugging Face today! Visit us at [huggingface.co](https://huggingface.co)

---
*Disclaimer: Not responsible for any AI creations that may take over the world. Hug at your own risk!*
```

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [43]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [44]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Welcome to Hugging Face! 🤗

## The AI Community Building the Future (and the Best Office Coffee)

At Hugging Face, we don't just build AI models; we build a community – a fantastic one where squishy robots and data scientists come together, love to collaborate, and curse at errors in unison. If you're looking to explore the exciting universe of models, datasets, and friendly AI enthusiasts, you've found your home!

---

### What do we do? 

- **Models Galore**: With **1+ million models** ready to go, we've got models for everything – from big ideas to tiny memes. Who needs Netflix when you can browse our models weekly? 

- **Datasets**: We've collected **250k+ datasets** that are more diverse than your last Tinder date. Whether you want to train a chatbot or just feel superior with large language model stats, we have everything you need (and more)!

- **Spaces**: Imagine a creative space with **10k+ AI applications** running smoother than your morning coffee. It’s more linked than your family reunion on a “bring your significant other” situation. 

---

### Why Hugging Face?

- **Spreading Love via Tech**: The AI community that collaborates (and hugs, figuratively speaking) to create fantastic tools for all. You won't just be another cog in the machine; you'll be part of a vibrant family of more than **50,000 organizations** worldwide, from Amazon to Zoom.

- **Open Source Philosophy**: Our motto is "more open, less closed!" Join the wave of developers who contribute to an ecosystem that values collaboration over competition. 

- **Career Opportunities**: Looking for a workplace that celebrates innovation and doesn’t mind the occasional dance-off during lunch breaks? **Careers at Hugging Face** offer the chance to work with **cutting-edge AI** while possibly performing the Macarena on Fridays. 

---

### Our Culture 
Hugging Face thrives on creativity, innovation, and, most importantly, a touch of humor. Our office culture is a mix of collaborative work, continuous learning, and spontaneous Nerf gun battles. If you chuckle at bad puns and love a good meme, you’re going to fit right in! 

### Join Us!
Whether you're looking to invest, collaborate, or find the perfect job that lets you geek out over AI, we've got opportunities waiting for you. Join us, and let’s change the world - one laugh, one model, and one hug at a time! 

---

**Contact Us**
- 📩 [Join the Community](https://huggingface.co/join)
- 🐦 Follow us on Twitter (@HuggingFace) for your weekly dose of AI musings and dad jokes.

Hugging Face: Where AI Meets Affection! 💖


In [45]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}]}



# Welcome to Hugging Face – Where AI Hugs You Back!

---

## Who Are We? 🤔
At Hugging Face, we don’t just build AI; we craft relationships! Think of us as the matchmaker of the machine learning world. We bring researchers, engineers, and data nerds together to create, collaborate, and occasionally share memes about neural networks!

---

## Our Offerings 🌟

### Models & Datasets 📊
Explore over **1 Million models** and **250k datasets**! Whether you need a model to understand cats or datasets about pizza preferences (hey, we all have our priorities), we’ve got it all! 

- Trending Models:
  - **moonshotai/Kimi-K2-Instruct** – The model updated just **25 minutes ago**, so fresh it might still be warm!
  - **black-forest-labs/FLUX.1-Kontext-dev** – With **254k** likes, this one is basically the Kardashian of AI models. 

### Spaces 🚀
We've created a platform where creativity has no bounds! Need to generate a dynamic video of a dancing cat? We call it "Cat-o-Graphy." 

---

## Join the Hugging Face Family! 🥰
We’re building more than just AI; we’re building a community! 

- **Team Culture**: Our office is a haven of collaboration filled with enthusiasts sharing knowledge like it’s candy on Halloween. Must have a sweet tooth for machine learning!
- **Careers**: Looking for a job where casual conversations about the latest AI breakthroughs happen over coffee? We might be the perfect fit for you! 

---

## Our Valued Customers 🏢
More than **50,000 organizations** are using Hugging Face, including giants like **Google, Microsoft**, and **Grammarly**. Apparently, even bots need a little help with their grammar!

---

## Let’s Build the Future Together! 🌍
Ready to join the most exciting AI community? Whether you’re a customer, investor, or future employee, there's a place for you at Hugging Face! 

---

### Laugh With Us 😂
In the words of our tech-savvy staff: “Why did the neural network go to therapy? Because it had too many layers!” 

---

## Contact Us 📞
- **Website**: [HuggingFace](https://huggingface.co)
- **Socials**: We’re lurking on Twitter, GitHub, and Discord! Come say hi! 

*Join us at Hugging Face – where we believe AI should give hugs, not just code!*



In [46]:
#kendim denedim
import os
from IPython.display import Markdown, display, update_display

def custom_brochure(company_name, url, stream=False, save_to_file=None):
    """
    Şirket broşürü oluşturur ve ekranda gösterir ya da dosyaya kaydeder.
    :param company_name: Şirket adı (örn. "HuggingFace")
    :param url: Web sitesi URL'si (örn. "https://huggingface.co")
    :param stream: True ise akış halinde, False ise toplu yanıt
    :param save_to_file: Dosya yolu, broşürü kaydetmek için (örn. "brochure.md")
    """
    try:
        # Kullanıcı mesajını al
        user_prompt = get_brochure_user_prompt(company_name, url)
        
        # OpenAI isteği
        if stream:
            # Akış modunda
            stream_response = openai.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                stream=True
            )
            response = ""
            display_handle = display(Markdown(""), display_id=True)
            for chunk in stream_response:
                chunk_content = chunk.choices[0].delta.content or ''
                response += chunk_content
                response = response.replace("```", "").replace("markdown", "")
                update_display(Markdown(response), display_id=display_handle.display_id)
        else:
            # Toplu yanıt modunda
            response = openai.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            ).choices[0].message.content
            display(Markdown(response))

        # Dosyaya kaydetme (isteğe bağlı)
        if save_to_file:
            with open(save_to_file, 'w', encoding='utf-8') as f:
                f.write(response)
            print(f"Broşür kaydedildi: {save_to_file}")

        return response

    except Exception as e:
        print(f"Hata oluştu: {str(e)}")
        return None

In [47]:
# Toplu yanıt, dosyaya kaydetmeden
custom_brochure("HuggingFace", "https://huggingface.co", stream=False)

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'forum page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


```markdown
# Welcome to Hugging Face! 🤗

## The AI Community Building the Future (One Hug at a Time)

Are you ready to embark on a journey into the heart of Artificial Intelligence? At Hugging Face, we believe that collaboration is the secret sauce to creating magical models, datasets, and applications – and we're serving it up with a side of humor!

### What We Do:

- **Models Galore**: With **1M+ models** to play with, our platform is a buffet of AI goodies. From moonshot models like **Kimi-K2-Instruct** to the popular **FLUX.1 Kontext**, there’s always something fresh out of the oven. Just don’t forget to bring your own chef’s hat!

- **Datasets**: Who knew datasets could be this exciting? Our collection of **250k+ datasets** is more delicious than any buffet. Got prompts? We’ve got the data!

- **Spaces**: Welcome to our playground! With **400k+ applications**, you can run everything from **DeepSite** to **ThinkSound** – like a theme park for data wizards!

### Our Culture: Hug Life 🤗

At Hugging Face, we embrace diversity like a cozy blanket – everyone is welcome. Our community is built on collaboration, curiosity, and the occasional dad joke. Yes, we acknowledge the punny AI chatbots making headlines!

- **Open Source Love**: Sharing is caring! We’re all about open-source contributions – because why keep the best models to ourselves? 

- **Team Spirit**: Whether remote or on-site, we’re like a team of superheroes fighting against the evil forces of dullness. Capes optional!

### Who Uses Hugging Face?

Join over **50,000 organizations** that trust us to power their AI endeavors! From **Meta** to **Microsoft**, the giants love our tools so much they probably send us thank-you notes (or at least, we like to think so).

### Careers at Hugging Face 🚀

Looking for a job where your ideas can take flight? Check out our **current openings**! We promise an environment where innovation blooms as freely as your coffee consumption. 

- **Current Opportunities**: Developers, data scientists, and anyone who loves AI – bring your dreams, and let’s build the future together!

- **Perks**: Work remotely, wear pajamas, and feel like a superhero. Plus, who can resist working for a company named **Hugging Face**? It's as comforting as it sounds!

### Join Us!

Become a part of our ever-growing community – whether you’re a developer, a data enthusiast or just someone who really, REALLY likes hugging (just kidding, we mean AI). 

So come on down to **Hugging Face** and remember, in the world of AI, don’t just be smart; be super-duper smart with us!

---

*Disclaimer: No actual "hugs" were given in the production of this brochure. Hugging Face, however, is a valid business name and a great place to grow your AI skills!*
```

'```markdown\n# Welcome to Hugging Face! 🤗\n\n## The AI Community Building the Future (One Hug at a Time)\n\nAre you ready to embark on a journey into the heart of Artificial Intelligence? At Hugging Face, we believe that collaboration is the secret sauce to creating magical models, datasets, and applications – and we\'re serving it up with a side of humor!\n\n### What We Do:\n\n- **Models Galore**: With **1M+ models** to play with, our platform is a buffet of AI goodies. From moonshot models like **Kimi-K2-Instruct** to the popular **FLUX.1 Kontext**, there’s always something fresh out of the oven. Just don’t forget to bring your own chef’s hat!\n\n- **Datasets**: Who knew datasets could be this exciting? Our collection of **250k+ datasets** is more delicious than any buffet. Got prompts? We’ve got the data!\n\n- **Spaces**: Welcome to our playground! With **400k+ applications**, you can run everything from **DeepSite** to **ThinkSound** – like a theme park for data wizards!\n\n### Ou

In [48]:
stream_brochure("Anthropic", "https://www.anthropic.com/")

Found links: {'links': [{'type': 'homepage', 'url': 'https://www.anthropic.com/'}, {'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}]}



# Welcome to Anthropic: Where AI Meets Good Vibes! 🌟

## Who Are We? 🤔

At Anthropic, we’re on a mission to build AI that serves humanity's long-term well-being! Think of us as the friendly neighborhood superheroes of Artificial Intelligence, swooping in to save the day (and your productivity) one *Claude* at a time!

## Meet Claude: Your AI Buddy! 🦸‍♂️

Say hello to Claude! No, he’s not just a French chef, but the name we've given to our brilliant AI models! We’ve got everything from **Claude Opus 4** – the brainiac of the group – to **Claude Sonnet 4**, who can rhyme better than your high school English teacher! Whether you want to chat, code, or brainstorm ideas, Claude is here to help with charm and wit!

## Our Culture: Zero Drama, Tons of Fantastics! 🎭

Here at Anthropic, we believe in a company culture as smooth as Claude's conversational maneuvers:
- **Transparency**: We're as clear as Claude’s ethical guidelines. No hidden agendas here!
- **Collaboration**: Our team works together like a well-oiled machine… or like Claude keeping a conversation flowing. 
- **Continuous Learning**: Join our **Anthropic Academy** where you can learn to build amazing things with Claude! (No, you won’t learn to make soufflés – sorry!)

## Who We Work With: You, Me, Everyone! 🎉

Whether you're a budding entrepreneur, a seasoned company, or someone who just wants to chat with a really smart AI, we’ve got you covered! Our customers love using Claude for everything from customer support to coding. Don’t worry, there are plenty of Claude-powered smiles to go around!

## Careers with Us: Join Our League of Extraordinary Programmers! 🚀

Want to help build the future of AI while having a stellar time? **Join our team!**
- We have open roles for all kinds of brilliant minds – whether you're a coding ninja or an AI philosopher!
- We believe good ideas come from people who have fun and express themselves. So bring your best ideas (and snacks)!

## Join Us! 🥳

If you’re ready to embark on an adventure to a future brimming with responsible AI, jump on board with Anthropic! Let’s innovate, create, and make the world a better place – one Claude at a time!

---

Let’s chat: **Talk to Claude** or **Explore our Career Opportunities** today! 



In [49]:
stream_brochure("VBAP", "https://www.vbap.com.tr/tr_TR/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.vbap.com.tr/tr_TR/kurumsal'}, {'type': 'homepage', 'url': 'https://www.vbap.com.tr/tr_TR'}, {'type': 'brochure page', 'url': 'https://www.vbap.com.tr/tr_TR/brosurler'}, {'type': 'careers page', 'url': 'https://www.vbap.com.tr/tr_TR/bize-ulasin'}, {'type': 'products page', 'url': 'https://www.vbap.com.tr/tr_TR/category/urunler'}]}



# Welcome to VBAP: Where SAP Meets Fun!

## 📞 Get in Touch
- Phone: +90 224 441 46 44
- Email: info@vbap.com.tr

---

### **Are You Ready to Upgrade Your Business?**

At VBAP, we believe that the only surprises you should have are the good ones! Let’s cut out those “bad surprise” budget meetings with our **SAP Integrated Budget Module**. Because who doesn’t love jaw-dropping moments… just not the kind that ruins your financial plan? 😱

### **Join the Digital Transformation Party! 🎉**

Say goodbye to messy warehouses! With our **Smart Warehouse Systems**, you'll be optimizing operations like a pro. Think of us as your magical assistants transforming clutter into order. Who needs a fairy godmother when you have VBAP? 🪄✨

### **More Than Just Software, It’s a Lifestyle! 😎**

At VBAP, we’re not just professionals; we’re passionate about turning businesses into well-oiled machines! With our ERP solutions, your operations will be smoother than a freshly waxed surfboard. 🏄‍♂️

---

## **Who Are Our Happy Clients? 🤔**

From automotive giants to savvy startups, we work with everyone. We're like the friendly neighbor lending tools, but in the world of tech consulting. Our clients trust us to steer their ships through the stormy seas of business management. 🌊⛵️

### **Join Our Team! 🌟**

Want to work with the coolest tech-savvy brains? VBAP isn't just about dazzling software solutions; it’s about groundbreaking ideas and laughter in the breakroom. Because what’s a good software without some fun? 

**Open Positions Available** – If you’re ready to mix innovation with a dash of humor, we need you on our crew! 🚀

---

### **The VBAP Culture: Inclusive, Innovative, and Just the Right Amount of Quirky! 🦄**

- **Talent Variety**: From number-crunching geniuses to creative wizards, our team is a vibrant patchwork of skills!
- **Work Hard, Joke Harder**: We believe in working diligently and celebrating even the smallest victories. You can count on us to “Excel” in humor, too! 😂

---

## **Ready to Transform Your Business?**

With *VBAP*, experiencing financial clarity and operational excellence is just a click away. Reach out today and join us in shaping the future of business technology!

---
**VBAP: Your Partner in Tech Solutions.** 
_Calling all dreamers, doers, and fun-lovers!_




<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

In [50]:
# Toplu yanıt, dosyaya kaydetmeden
custom_brochure("EliteCunstruction", "https://www.eliteconstructionde.com/", stream=False)

Found links: {'links': [{'type': 'about page', 'url': 'https://www.eliteconstructionde.com/about/'}, {'type': 'contact page', 'url': 'https://www.eliteconstructionde.com/contact/'}]}


# 🏠 Welcome to EliteCunstruction! 🏠

## Your Home’s Makeover Dream Team!

---

### Have a Project in Mind? Let’s Get Started!

* **Services We Offer:**
  - Kitchen & Bath Remodeling (Because your kitchen shouldn’t just be for food storage!)
  - Room Additions (More space for your growing collection of... well, things!)
  - Basement Finishing (Because we all know it’s the heart of the home... or maybe it just needs a little help!)
  - Deck Installation & General Repairs (Perfect for entertaining all your “invisible” friends!)

---

### Walls Out, Worries Gone! 

With a **4.70/5.00** rating from over **24 Happy Customers** (and counting!), we’re here to make your remodeling dreams come true! We pride ourselves on clear communication—because nothing says "trust me" like a contractor who doesn’t speak in mysterious construction jargon.

### The EliteCunstruction Culture 

We believe in collaboration, creativity, and not leaving our tools in your living room just because it’s “super convenient.” With us, you get a team that cares about quality, respects your home, and—most importantly—will show up to finish the job (bonus points if we show up on time)!

---

### Our Happy Customers

From Lewes to Milton, we steal hearts—and spaces! Our customers are thrilled with the transformations that turn their homes from “meh” to “wow!” We may not have magic wands, but our hammer can do wonders!

---

### Want to Join the Fun?

**Careers at EliteCunstruction:**
Are you a handyman with a sense of humor? A designer with an eye for detail? Or a superhero with a tool belt? We’re always on the lookout for talented minds and friendly faces to join our crew. Our culture values creativity, communication, and caffeination. If you think you’ve got what it takes to help us transform homes, we might just have the perfect spot for you!

---

### Get in Touch!

Ready to give your home the makeover it deserves? Or curious about becoming part of our elite team? Don’t hesitate! Reach out and let’s bring your ideas to life—preferably before our coffee cools down! 

* **Contact Us:** [EliteCunstruction.com/contact](#) 
* **Your dream home is just a click away!**

---

**EliteCunstruction: Because Every Home Needs a Bit of Love (and a Lot of Remodeling)!**

'# 🏠 Welcome to EliteCunstruction! 🏠\n\n## Your Home’s Makeover Dream Team!\n\n---\n\n### Have a Project in Mind? Let’s Get Started!\n\n* **Services We Offer:**\n  - Kitchen & Bath Remodeling (Because your kitchen shouldn’t just be for food storage!)\n  - Room Additions (More space for your growing collection of... well, things!)\n  - Basement Finishing (Because we all know it’s the heart of the home... or maybe it just needs a little help!)\n  - Deck Installation & General Repairs (Perfect for entertaining all your “invisible” friends!)\n\n---\n\n### Walls Out, Worries Gone! \n\nWith a **4.70/5.00** rating from over **24 Happy Customers** (and counting!), we’re here to make your remodeling dreams come true! We pride ourselves on clear communication—because nothing says "trust me" like a contractor who doesn’t speak in mysterious construction jargon.\n\n### The EliteCunstruction Culture \n\nWe believe in collaboration, creativity, and not leaving our tools in your living room just beca